# Assignment 3
## Bulut Fıçıcı

## Tried to implement the Watkin's Q Lambda algorthym but continuous states were the main problem faced with.  

## The article below gave a Cart-Pole example with Q Lambda, but it includes discretization of the continuous state variables. However, pseudo-code in the book or the Chapter 9 does not contain discretization process.

http://karanmg.com/Computers/reinforcementLearning/finalProject/KaranComparisonOfSarsaWatkins.pdf

## Tried different approaches but couldn't find a solution for the problem. Hence tried to gave you a full solution.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.kernel_approximation import RBFSampler
from sklearn.preprocessing import PolynomialFeatures
import time
import gym

In [2]:
env = gym.make("CartPole-v0")
# initial state
s=env.reset()
s

array([ 0.04057275, -0.0357134 , -0.0364759 ,  0.0430213 ])

In [3]:
env.reset()
done=False
while not done:
    a=env.action_space.sample() #random action
    s_next, r, done, info = env.step(a)
    time.sleep(0.05)
    env.render()
env.close()

In [4]:
def merge_state_action(s, a):
    return np.concatenate((s, [a]))

In [5]:
s=env.reset()
a=env.action_space.sample()
sa=merge_state_action(s, a)
sa

array([ 0.02253938, -0.04260251, -0.03214092,  0.03729644,  1.        ])

In [6]:
def gather_samples(n_episodes=10000):
    samples = []
    for i in range(n_episodes):
        s = env.reset()
        done=False
        while not done:
            a=env.action_space.sample() #random action
            sa = merge_state_action(s, a)
            samples.append(sa)
            s, r, done, info = env.step(a)
    return samples

In [7]:
samples=gather_samples(n_episodes=10000)
samples

[array([0.02944272, 0.00727601, 0.02959009, 0.02038687, 0.        ]),
 array([ 0.02958824, -0.18825752,  0.02999783,  0.32225699,  1.        ]),
 array([0.02582309, 0.0064247 , 0.03644297, 0.03918308, 1.        ]),
 array([ 0.02595158,  0.20100562,  0.03722663, -0.24178265,  0.        ]),
 array([0.02997169, 0.00537223, 0.03239098, 0.06240623, 1.        ]),
 array([ 0.03007914,  0.20001516,  0.0336391 , -0.21988373,  0.        ]),
 array([0.03407944, 0.00442891, 0.02924143, 0.08321769, 0.        ]),
 array([ 0.03416802, -0.19109974,  0.03090578,  0.38498104,  1.        ]),
 array([0.03034602, 0.00357013, 0.0386054 , 0.10220053, 0.        ]),
 array([ 0.03041743, -0.19208321,  0.04064941,  0.40680904,  0.        ]),
 array([ 0.02657576, -0.38775729,  0.04878559,  0.71202533,  0.        ]),
 array([ 0.01882062, -0.5835196 ,  0.0630261 ,  1.0196567 ,  0.        ]),
 array([ 0.00715022, -0.77942229,  0.08341923,  1.33144471,  0.        ]),
 array([-0.00843822, -0.97549135,  0.11004813,  1.

In [8]:
rbf_feature = RBFSampler(n_components=100)
rbf_feature.fit(samples)

RBFSampler()

In [9]:
def predictV(s,a,w,kernel):
    sa = merge_state_action(s, a)
    if kernel=='poly':
        poly_features.fit([sa])
        x=poly_features.transform([sa])[0]
    elif kernel=='rbf':
        x = rbf_feature.transform([sa])[0]
    else:
        x=nystrom_featurizer.transform([sa])[0]
    return np.dot(x,w)

In [10]:
def gradientV(s,a,kernel):
    sa = merge_state_action(s, a)
    if kernel=='poly':
        poly_features.fit([sa])
        x=poly_features.transform([sa])[0]
    elif kernel=='rbf':
        x = rbf_feature.transform([sa])[0]
    else:
        x=nystrom_featurizer.transform([sa])[0]
    return x

In [11]:
def predictV_all_actions(s,w,kernel):
    values=[]
    for a in range(env.action_space.n):
        values.append(predictV(s,a,w,kernel))
    return values

In [12]:
def epsilon_greedy(s, w, kernel, eps=0.1):
    values=[]
    p = np.random.random()
    if p < (1 - eps):
        values = predictV_all_actions(s,w,kernel)
        return np.argmax(values)
    else:
        return env.action_space.sample()

In [13]:
def watch_agent(w,kernel):
    done = False
    episode_reward = 0
    s = env.reset()
    while not done:
        a = epsilon_greedy(s, w, kernel,eps=0)
        s, r, done, info = env.step(a)
        time.sleep(0.05)
        env.render()
        episode_reward += r
    print("Episode reward:", episode_reward)

In [14]:
GAMMA=1
ALPHA = 0.1
w = np.zeros(100) 
n_episodes = 500
for it in range(n_episodes):
    s=env.reset()
    done=False
    while not done:
        a = epsilon_greedy(s, w, kernel='rbf',eps=0.1)
        s2, r, done, info = env.step(a)
        if done:
            target = r
        else:
            values = predictV_all_actions(s2,w,kernel='rbf')
            target = r + GAMMA * np.max(values)               
        g = gradientV(s,a,kernel='rbf')
        err=target - predictV(s,a,w,'rbf')
        w += ALPHA * err * g
        # update state
        s = s2

In [15]:
watch_agent(w,kernel='rbf')
env.close()

Episode reward: 45.0
